In [1]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, make_scorer
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import RepeatedStratifiedKFold
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline

In [2]:
# LEER LOS DATOS 
file_in = pd.read_csv('dead_due_to_AHE.csv') #lee archivo 
file_in = file_in.drop(['subject_id', 'hadm_id', 'icustay_id'], axis= 'columns') #elimino las columnas que no ocupo 

# IMPUTACIÓN DE DATOS 
x = file_in.drop(['dead'], axis='columns') #para mis variables de entrada soalamente elimino la varaible a predecir
y= file_in.dead #me quedo con la variable a predecir en y 
columnas= x.columns 


# DIVIDIR CONJUNTO DE ENTRENAMIENTO, VALIDACIÓN Y PRUEBA 
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, )
print('X_train:', x_train.shape, 'X_test:', x_test.shape) 

#data = x.replace(" ", np.NaN)
#values = data.values
#imp = IterativeImputer(max_iter=10, random_state=0)
#imp = KNNImputer(n_neighbors=5, weights= 'uniform')
##imp = MissForest()

imp= SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(x_train) #mean, most frequent, constant 
x_train = imp.transform(x_train)
x_test = imp.transform(x_test)
print('X_train:', x_train.shape, 'X_test:', x_test.shape)

X_train: (3563, 40) X_test: (1528, 40)
X_train: (3563, 40) X_test: (1528, 40)


In [3]:
print(type(x_train))
x_train_imputado = pd.DataFrame(x_train, columns=[columnas]) #convirtiendo el arreglo en dataframe 
x_test_imputado = pd.DataFrame(x_test, columns=[columnas])
x_train_imputado

<class 'numpy.ndarray'>


,age,kg,creatinine,sexo,first_sicu,first_micu,first_ccu,first_tsicu,first_csru,first_nicu,...,resprate_max,tempc_min,tempc_mean,tempc_max,spo2_min,spo2_mean,spo2_max,pao2_min,pao2_mean,pao2_max
0,61.0,71.900002,1.4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,30.0,34.166667,35.600001,36.000002,87.0,94.290323,100.0,46.0,67.666667,110.000000
1,34.0,73.000000,0.8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,20.0,37.111113,37.349207,37.500000,95.0,97.521739,100.0,88.0,116.000000,144.000000
2,55.0,66.699997,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,31.0,33.999998,36.163889,37.777778,91.0,96.604938,100.0,66.0,92.500000,164.000000
3,81.0,88.300003,2.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,34.0,34.333335,35.484331,37.222222,86.0,94.986014,100.0,61.0,81.058824,146.000000
4,68.0,81.000000,0.9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,28.0,36.055556,37.456141,38.611111,90.0,94.282051,98.0,69.0,88.000000,415.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558,49.0,82.900002,0.8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,36.000000,37.095437,38.000000,85.0,97.139535,100.0,82.0,165.615385,471.000000
3559,64.0,118.000000,2.3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,29.0,36.055556,37.320455,38.500000,89.0,96.401786,100.0,72.0,158.625000,438.000000
3560,66.0,102.500000,0.7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,34.200001,36.914074,38.299999,94.0,96.980198,100.0,70.0,199.459091,448.100006
3561,74.0,87.199997,0.8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,26.0,35.000000,36.363027,37.700001,92.0,99.085366,100.0,126.0,290.285714,450.000000


In [4]:
# ESTANDARIZACIÓN DE DATOS 
estandarizar = ['age', 'kg', 'creatinine','heartrate_min', 'heartrate_mean', 'heartrate_max', 'sysbp_min',
       'sysbp_mean', 'sysbp_max', 'diasbp_min', 'diasbp_mean', 'diasbp_max',
       'meanbp_min', 'meanbp_mean', 'meanbp_max', 'resprate_min','resprate_mean', 'resprate_max', 'tempc_min', 'tempc_mean', 'tempc_max',
       'spo2_min', 'spo2_mean', 'spo2_max', 'pao2_min', 'pao2_mean','pao2_max']
dummies = ['sexo', 'first_sicu', 'first_micu',
       'first_ccu', 'first_tsicu', 'first_csru', 'first_nicu', 'last_sicu','last_micu', 'last_ccu', 'last_tsicu', 'last_csru', 'last_nicu']

#Intentar con las varaibles separadas y también juntas 
scaler = StandardScaler().fit(x_train_imputado)
#transformed_values[estandarizar] = scaler.fit_transform(transformed_values[estandarizar]) #estandarizo las variables númericas 
x_train_escalado = scaler.fit_transform(x_train_imputado)
x_test_escalado = scaler.fit_transform(x_test_imputado)

#COLOCAR PRINTS 
print('X_train:', x_train_escalado.shape, 'X_test:', x_test_escalado.shape)

X_train: (3563, 40) X_test: (1528, 40)


In [5]:
# DEFINIR MODELO A UTILIZAR 
#RF
modeloXGB = xgb.XGBClassifier(use_label_encoder=False)
modeloRF = RandomForestClassifier()
modeloSVM = svm.LinearSVC()

#DEFINIR PARÁMETROS A ITERAR

parametrosRF = {'n_estimators': range(10, 200, 10), 'criterion':['gini', 'entropy'], 'max_depth': range(1,10,1), 
              'max_features':['auto', 'sqrt', 'log2'],  'min_impurity_decrease':[1e-4, 1e-7], 
              'class_weight':['balanced', 'balanced_subsample']} # PARA RF

parametrosRFprueba = {'n_estimators': range(10, 200, 10),'max_depth': range(1,10,1),'class_weight':['balanced']}

ratio = float(np.sum(y_train == 0)) / np.sum(y_train==1)
                      
parametrosXGB = {'loss':['deviance', 'exponential'], 'criterion':['friedman_mse', 'mse', 'mae'], 'max_features':['auto', 'sqrt',
              'log2'], 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05],"colsample_bytree" : [0.4, 0.8],
                    "subsample" : [0.8, 0.4], "reg_alpha" : [0.5], "reg_lambda": [2],
                    "objective": ['binary:logistic'],  'max_depth': range(1,10,1), "gamma":[10],"rate_drop": [0.5, 0.3],
                    "seed": [422], "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosXGBprueba = { 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05], 'max_depth': range(1,10,1),
                        "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosSVM = {'C':np.logspace(-3,3,7), 'loss':['hinge', 'squared_hinge'], 'multi_class':['ovr', 'crammer_singer'],
                       "class_weight": ['balanced']}

#ENTRENAR GRID SEARCH 
#TIPOS DE SCORE 
balanced = class_weight.compute_sample_weight('balanced', y_train)
RKF = RepeatedKFold(n_splits=5, n_repeats=2, random_state=422) #probar los CROSS VALIDATION
SKF = StratifiedKFold(n_splits=5, shuffle= True, random_state=422)
RSKF = RepeatedStratifiedKFold(n_splits=10, n_repeats=10)
grid = GridSearchCV(estimator= modeloSVM, param_grid= parametrosSVM, cv=RSKF, scoring='roc_auc', verbose=2, refit= True)

In [6]:
grid.fit(x_train, y_train, sample_weight = balanced)

Fitting 100 folds for each of 28 candidates, totalling 2800 fits


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END C=0.001, class_weight=balanced, loss=hinge, multi_class=ovr; total time=   0.2s


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END C=0.001, class_weight=balanced, loss=hinge, multi_class=ovr; total time=   0.2s


...

C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END C=0.1, class_weight=balanced, loss=squared_hinge, multi_class=crammer_singer; total time= 1.6min


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END C=0.1, class_weight=balanced, loss=squared_hinge, multi_class=crammer_singer; total time= 1.6min


KeyboardInterrupt: 

In [ ]:
# METRICAS DEL MODELO 
auc_train = grid.best_score_
std_train = grid.cv_results_['std_test_score'][grid.best_index_]
best_params = grid.best_params_
best_estimator = grid.best_estimator_
print("auc:",auc_train)
print("std_validation:",std_train)
print('best parameters:',best_params)
print('best estimator:', best_estimator)